In [ ]:
import math
import numpy as np
import pickle
import sys

sys.path.append('..')
import unzipping_simulation as uzsi

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the parameters of the DNA unzipping construct
with open('phage_lambda.fasta') as f:
    sequence = ''.join([line.rstrip() for line in f if not line.startswith('>')])
bases = sequence[39302:40041]
# complementary linker sequence, primer, bases, primer, hairpin (polyT left out)
bases = 'GATACGTTCTTACCCATACTCCACCGTTGC' + 'TGTGCCAACA' + 'CATTGC' + bases + 'GCAATG' + 'CAAGCTACTG' + 'CCGGTCGTAT'
nbs = 10  # 2 * 5pT
nbs_loop = 10  # hairpin with 10 pT loop
seg_a = 42588 - 42168 + 1 + 2*(8 + 10)
seg_b = 43761 - 42854 + 1 + (6 + 10) + (8 + 10)
nbp = seg_a + seg_b  # (1399)

# Set the parameters of the optical tweezers setup/assay
radius = 410e-9
r = radius
# distance between surface of the bead and the glass
h0 = 320e-9
z0 = h0
# 3D positioning
y0 = 0e-9
# A0 = attachment_point(x0, y0=y0, h0=h0, radius=radius)

# Stiffness
kappa = np.array([0.67058388e-3, 0.59549569e-3, 0.20775878e-3])

T = 298.2

# Set stage displacement to be simulated
x0_min = 100e-9
x0_max = 1550e-9
resolution = 5e-9

# Set the boltzmann factor for selection of the probable states
# used to determine the simulated forces/unzipped basepairs
boltzmann_factor = 1e-5

# Set the parameters for the ssDNA model 
# Elastic modulus S, literature values are
# in the range of 0.53 ≤ K ≤ 2.2 nN 
S = 840e-12
# Persistence length 0.75 ≤ L_p ≤ 3.1 nm
L_p_ssDNA = 0.797e-9
# Contour length per base 0.43 ≤ L_0 ≤ 0.66 nm/base
# higher values stretch the unzipping curve in x
# The influence of z increases for higher numbers of unzipped basepairs
# (-> longer ssDNA -> more influence on curve)
z = 0.568e-9

# Set the parameters for the dsDNA model
pitch = 0.338e-9
L_p_dsDNA = 50e-9

# Use Nearest neighbour base-pairs for calculation of the unpairing energies?
NNBP = True
# Set the concentration of monovalent cations
c = 50e-3

# The free energy of opening the terminal hairpin in kcal/mol
e_loop = 0.0

simulation = {}
simulation['settings'] = {
    'x0_min': x0_min,
    'x0_max': x0_max,
    'y0': y0,
    'h0': h0,
    'resolution': resolution,
    'bases': bases,
    'nbs': nbs,
    'nbp': nbp,
    'nbs_loop': nbs_loop,
    'radius': radius,
    'kappa': kappa,
    'S': S,
    'L_p_ssDNA': L_p_ssDNA,
    'z': z,
    'pitch': pitch,
    'L_p_dsDNA': L_p_dsDNA,
    'NNBP': NNBP,
    'c': c,
    'e_loop': e_loop,
    'T': T,
    'boltzmann_factor': boltzmann_factor
}

# Save simulation parameters
settings_file = 'simulation_settings.p'

with open(settings_file,'wb') as f:
    pickle.dump(simulation, f)